# Dec 6, 19, 2024: comparing communities with RSNs/ICs
- does any community resemble a known ic?
- does a group of comms resemble a known rsn?
- what is the natural grouping of comms that resemble a group of ics?

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats, linalg
from scipy.spatial.distance import jensenshannon, cosine
from multiprocessing import Pool
import glob
import random

from sklearn.cluster import DBSCAN, SpectralCoclustering, SpectralClustering
from scipy.optimize import linear_sum_assignment

import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
RSN_ROI_path = f'{ROI_path}/rsns'
os.system(f'mkdir -p {RSN_ROI_path}')
IC_ROI_path = f'{ROI_path}/ics'
os.system(f'mkdir -p {IC_ROI_path}')
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [6]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids

marginals_df

,sub,sbm,pi_0,pi_1,pi_2,pi_3,pi_4,pi_5,pi_6,pi_7,pi_8,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.08362369337979095, 0.0, 0.0034843205574912...","[[0.7979094076655052, 0.017421602787456445, 0....","[[0.49477351916376305, 0.0, 0.3205574912891986...","[[0.926829268292683, 0.05574912891986063, 0.0,...","[[0.9860627177700348, 0.010452961672473868, 0....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,0.286725,0
1,SLC01,sbm-nd-h,"[[0.06274509803921569, 0.0, 0.0, 0.0, 0.360784...","[[0.8705882352941177, 0.0196078431372549, 0.06...","[[0.5294117647058824, 0.0, 0.30980392156862746...","[[0.9529411764705882, 0.03529411764705882, 0.0...","[[0.9921568627450981, 0.00392156862745098, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,0.254538,1
2,SLC01,sbm-nd-h,"[[0.0196078431372549, 0.0, 0.0, 0.0, 0.3431372...","[[0.9901960784313726, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.8529411764705882, 0.0, 0.10784313725490197...","[[0.9803921568627451, 0.0196078431372549], [0....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.102439,2
3,SLC01,sbm-nd-h,"[[0.02531645569620253, 0.0, 0.0, 0.0, 0.316455...","[[0.9620253164556962, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.7468354430379747, 0.0, 0.20253164556962025...","[[0.9746835443037974, 0.02531645569620253], [1...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.079128,3
4,SLC01,sbm-nd-h,"[[0.07692307692307693, 0.0, 0.0, 0.0, 0.320512...","[[0.7948717948717948, 0.07692307692307693, 0.0...","[[0.7692307692307693, 0.0, 0.15384615384615385...","[[0.9615384615384616, 0.038461538461538464], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.078369,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,SLC10,sbm-nd-h,"[[0.02531645569620253, 0.0, 0.0, 0.0, 0.316455...","[[0.9620253164556962, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.7468354430379747, 0.0, 0.20253164556962025...","[[0.9746835443037974, 0.02531645569620253], [1...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.080736,3
70,SLC10,sbm-nd-h,"[[0.07692307692307693, 0.0, 0.0, 0.0, 0.320512...","[[0.7948717948717948, 0.07692307692307693, 0.0...","[[0.7692307692307693, 0.0, 0.15384615384615385...","[[0.9615384615384616, 0.038461538461538464], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.052651,4
71,SLC10,sbm-nd-h,"[[0.22666666666666666, 0.0, 0.0, 0.0, 0.133333...","[[0.8133333333333334, 0.06666666666666667, 0.0...","[[0.5733333333333334, 0.0, 0.26666666666666666...","[[0.9466666666666667, 0.05333333333333334], [0...","[[0.9733333333333334, 0.02666666666666667], [0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,0.035607,5
72,SLC10,sbm-nd-h,"[[0.09375, 0.0, 0.0, 0.0, 0.078125, 0.0, 0.765...","[[0.734375, 0.0625, 0.078125, 0.0, 0.0, 0.0, 0...","[[0.828125, 0.0, 0.171875], [0.921875, 0.0, 0....","[[0.953125, 0.046875], [0.984375, 0.015625], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",NaN,NaN,NaN,NaN,0.017924,6


In [7]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0', 'pi_1', 'pi_2', 'pi_3', 'pi_4', 'pi_5', 'pi_6', 'pi_7', 'pi_8']

---

similarities between ics/rsns and comms

In [8]:
nii_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/marginal-visuals/nii/*', recursive=True))
rsn_files = sorted(glob.glob(f'{RSN_ROI_path}/*-mask.nii.gz', recursive=True))
ic_files = sorted(glob.glob(f'{IC_ROI_path}/*-mask*.nii.gz', recursive=True))

In [9]:
rsn_files

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-basal_ganglia-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-limbic-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-olfactory-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-sensory-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-somatosensory-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/rsns/desc-j-visual-mask.nii.gz']

In [10]:
ic_files

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-00-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-01-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-02-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-03-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-04-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-05-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/ics/desc-j-06-mask.nii.gz',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-

In [11]:
def get_mode_and_level(nii_file):
    mode_level = [n for n in nii_file.split('/') if 'mode-' in n][0].split('.')[0]
    if len(mode_level.split('_')) == 2:
        mode, level = mode_level.split('_')
        mode = int(mode.split('-')[-1])
        level = int(level.split('-')[-1])
        col = f'pi_{level}'
    elif len(mode_level.split('_')) == 1:
        mode = mode_level
        mode = int(mode.split('-')[-1])
        level = 'aligned'
        col = f'pi_{level}'
    
    return col, mode

In [12]:
# similarity matrices (based on distance between 3D shapes)

In [13]:
def find_similarity(comm_vol, ic_vol):
    if np.array_equal(np.unique(comm_vol), [0.0]):
        dist = 1.0
    else:
        dist = cosine(comm_vol.flatten(), ic_vol.flatten())
    sim = 1 - dist
    return sim

def find_similarities_per_sys(nii_vol, ic_vol):
    sims = []
    num_comms = nii_vol.shape[-1] if len(nii_vol.shape) == 4 else 1
    for comm in range(num_comms):
        comm_vol = nii_vol[..., comm] if num_comms > 1 else nii_vol
        sim = find_similarity(comm_vol, ic_vol)
        sims += [sim]
    return np.array(sims)

def fill_similarity_matrix(nii_vol, ic_files):
    # similarity matrix
    num_comms = nii_vol.shape[-1] if len(nii_vol.shape) == 4 else 1
    X = np.zeros((num_comms, len(ic_files))) # num_comms x num_rsns
    for ic, ic_file in enumerate(ic_files):
        ic_vol = ants.image_read(ic_file).numpy()
        X[:, ic] = find_similarities_per_sys(nii_vol, ic_vol)
    return X

In [14]:
def get_similarity_matrix(nii_files, ic_files):
    similarities_df = marginals_df.copy(deep=True)
    for nii_file in tqdm(nii_files):
        sub = [n for n in nii_file.split('/') if 'sub-' in n][0].split('-')[-1]
        col, mode_id = get_mode_and_level(nii_file)

        row = similarities_df[
            (similarities_df['sub'] == sub) &
            (similarities_df['mode_id'] == mode_id)
        ]
        idx = row.index.to_list()[0]

        assert(row['sub'].to_list()[0] == sub)
        assert(row['mode_id'].to_list()[0] == mode_id)

        nii_vol = ants.image_read(nii_file).numpy()
        X = fill_similarity_matrix(nii_vol, ic_files)
        similarities_df.at[idx, col] = X.T # num_ics x num_comms
    return similarities_df

In [15]:
similarities_ics_df = get_similarity_matrix(nii_files, ic_files)
similarities_ics_df

100%|██████████| 403/403 [03:53<00:00,  1.73it/s]


,sub,sbm,pi_0,pi_1,pi_2,pi_3,pi_4,pi_5,pi_6,pi_7,pi_8,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.09725113213062286, 0.0, 0.3025919795036316...","[[0.4153113067150116, 0.004015007056295872, 0....","[[0.26607105135917664, 0.002108269138261676, 0...","[[0.35421040654182434, 0.45741361379623413, 0....","[[0.3596718907356262, 0.45619282126426697, 0.0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,0.286725,0
1,SLC01,sbm-nd-h,"[[0.12153148651123047, 0.0, 0.2437444031238556...","[[0.4664391875267029, 0.00719763059169054, 0.2...","[[0.2757693827152252, 0.002572651021182537, 0....","[[0.35566094517707825, 0.4340307414531708, 0.0...","[[0.36044901609420776, 0.3467515707015991, 0.0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,0.254538,1
2,SLC01,sbm-nd-h,"[[0.12321822345256805, 0.0, 0.2456046342849731...","[[0.4868360757827759, 0.007680839858949184, 0....","[[0.3433952331542969, 0.0, 0.4218049645423889,...","[[0.3583933413028717, 0.47288671135902405], [0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.102439,2
3,SLC01,sbm-nd-h,"[[0.12144186347723007, 0.0, 0.2392457574605941...","[[0.49435341358184814, 0.005159799940884113, 0...","[[0.32319071888923645, 0.0, 0.4527735710144043...","[[0.35657623410224915, 0.4955541789531708], [0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.079128,3
4,SLC01,sbm-nd-h,"[[0.11846701800823212, 0.0, 0.2427757680416107...","[[0.4539813697338104, 0.006052723620086908, 0....","[[0.3352108895778656, 0.0, 0.4214757978916168,...","[[0.35619157552719116, 0.4723048806190491], [0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.078369,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,SLC10,sbm-nd-h,"[[0.12144186347723007, 0.0, 0.2392457574605941...","[[0.49435341358184814, 0.005159799940884113, 0...","[[0.32319071888923645, 0.0, 0.4527735710144043...","[[0.35657623410224915, 0.4955541789531708], [0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.080736,3
70,SLC10,sbm-nd-h,"[[0.11846701800823212, 0.0, 0.2427757680416107...","[[0.4539813697338104, 0.006052723620086908, 0....","[[0.3352108895778656, 0.0, 0.4214757978916168,...","[[0.35619157552719116, 0.4723048806190491], [0...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.052651,4
71,SLC10,sbm-nd-h,"[[0.115414097905159, 0.0, 0.2915729284286499, ...","[[0.4769476354122162, 0.0008495661895722151, 0...","[[0.2890530526638031, 0.0, 0.4535020589828491,...","[[0.35322079062461853, 0.4974314868450165], [0...","[[0.356149286031723, 0.5327690839767456], [0.2...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,0.035607,5
72,SLC10,sbm-nd-h,"[[0.09905298054218292, 0.0, 0.3016144633293152...","[[0.48571887612342834, 0.02946028858423233, 0....","[[0.337923526763916, 0.0, 0.47628864645957947]...","[[0.35417211055755615, 0.47300398349761963], [...","[[0.3609345853328705], [0.27063363790512085], ...",NaN,NaN,NaN,NaN,0.017924,6


In [16]:
for sub, group in similarities_ics_df.groupby('sub'):
    folder = f'{ESTIM_path}/individual/sub-{sub}/comparions-with-known-systems/{SBM}'
    os.system(f'mkdir -p {folder}')

    with open(f'{folder}/sys-ics_desc-similarities.pkl', 'wb') as f:
        pickle.dump(group, f)

In [17]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [18]:
def get_soft_similarity_matrix(similarities_df, num_sys):
    soft_similarities_df = []
    for sub, group in similarities_df.groupby('sub'):
        omegas = group['omega'].to_numpy()
        dct = {'sub': [sub], 'sbm': [SBM]}
        for col in cols:
            M = get_membership_matrix(num_sys, group, col)
            # num_rsns x num_modes x num_comms

            soft_X = np.average(M, axis=1, weights=omegas)
            dct[col] = [soft_X]
        soft_similarities_df += [pd.DataFrame(dct)]
    soft_similarities_df = pd.concat(soft_similarities_df).reset_index(drop=True)
    return soft_similarities_df

In [19]:
soft_similarities_ics_df = get_soft_similarity_matrix(similarities_ics_df, len(ic_files))
soft_similarities_ics_df

,sub,sbm,pi_0,pi_1,pi_2,pi_3,pi_4,pi_5,pi_6,pi_7,pi_8
0,SLC01,sbm-nd-h,"[[0.11286468472106845, 0.0, 0.2672867883382583...","[[0.4585637658715296, 0.0071530334261774445, 0...","[[0.2971784840150625, 0.0012593320428357661, 0...","[[0.3556856126558395, 0.4513541560240243, 0.0,...","[[0.360088849626556, 0.2591543637111825, 0.0, ...","[[0.22252100644732228], [0.16684926280923865],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
1,SLC02,sbm-nd-h,"[[0.11339037476693818, 7.474421858429695e-08, ...","[[0.4602513063901039, 0.006710608624086256, 0....","[[0.3034929142836597, 0.0009896938773947483, 0...","[[0.355855676601571, 0.4589574867566872, 0.0, ...","[[0.3601545906911403, 0.2203671228809311, 1.18...","[[0.18363820087058036], [0.13769440884730424],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2,SLC03,sbm-nd-h,"[[0.10816944488932499, 0.0, 0.2769693162625349...","[[0.44205371548376415, 0.005422928639182535, 0...","[[0.2832142870243513, 0.0017957750719508623, 0...","[[0.35533380066664083, 0.4557118503397335, 0.0...","[[0.36010978773585034, 0.33869467363116224, 0....","[[0.2885455044657499], [0.21635532516983896], ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
3,SLC04,sbm-nd-h,"[[0.11248267117917539, 0.0, 0.2682027612793446...","[[0.45477873278975495, 0.00515709917549044, 0....","[[0.2958010380804539, 0.0011880613548122347, 0...","[[0.3554681680500508, 0.4657213314974308, 0.0,...","[[0.35993787272095684, 0.2720178210139275, 0.0...","[[0.2249199961960316], [0.16864805779695513], ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
4,SLC05,sbm-nd-h,"[[0.10996055490777093, 0.0, 0.2716124344138992...","[[0.44682458995080676, 0.005472374443070548, 0...","[[0.29001503576044557, 0.0015368967036363253, ...","[[0.35549995822698677, 0.4596123179820765, 0.0...","[[0.3602316908362577, 0.2892716490676621, 0.0,...","[[0.24672281770694737], [0.1849961085016372], ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
5,SLC06,sbm-nd-h,"[[0.12178051626266155, 2.1861884453275968e-05,...","[[0.4668136206203089, 0.007961876548053167, 0....","[[0.3112229902855418, 0.0007415542840770591, 0...","[[0.3563350594658974, 0.44525485050203695, 0.0...","[[0.3602414001701211, 0.18884725120037069, 0.0...","[[0.17750018138626997, 0.0], [0.13309203834861...","[[0.009573899983024327], [0.007178639805204843...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
6,SLC07,sbm-nd-h,"[[0.10963778936717598, 0.0, 0.2730075541160689...","[[0.4461803563899716, 0.005328730946758922, 0....","[[0.2891433651712259, 0.0015273386850065524, 0...","[[0.3553998554751274, 0.46019457386847185, 0.0...","[[0.3601300881560888, 0.30002490174596413, 0.0...","[[0.25318085823849207], [0.18983843471201897],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
7,SLC08,sbm-nd-h,"[[0.1130544260343869, 0.0, 0.26707152403657186...","[[0.46130890478922876, 0.007142101268192212, 0...","[[0.30095415652644464, 0.0010590758417032614, ...","[[0.3555104244019932, 0.4663061828666676, 0.0,...","[[0.3600605642490448, 0.23693745363547644, 0.0...","[[0.19941256934107598], [0.14952224385759824],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
8,SLC09,sbm-nd-h,"[[0.1114249814026168, 0.0, 0.2708594721431782,..

In [20]:
folder = f'{ESTIM_path}/group/comparisons-with-known-systems/{SBM}'
os.system(f'mkdir -p {folder}')

for col in cols:

    ncols = 1
    nrows = len(soft_similarities_ics_df)
    fig, axs = plt.subplots(nrows, ncols, figsize=(20*ncols, 6*nrows),)
    fig.tight_layout(h_pad=3, w_pad=3)
    fig.suptitle(f'{SBM} {col}', x=0.0, y=1.0)
    for idx, row in soft_similarities_ics_df.iterrows():
        r = idx
        ax = axs[r]
        sub = row['sub']
        soft_X = row[col]
        sns.heatmap(soft_X, ax=ax, vmin=0.0, vmax=1.0, annot=True, fmt='.2f')
        ax.set(xlabel=f'comm', ylabel=f'ic', title=f'{sub}: similarity matrix')

    fig.savefig(f'{folder}/col-{col}_sys-ic_desc-similarity-matrices.pdf', bbox_inches='tight')
    plt.close('all')

In [21]:
similarities_rsns_df = get_similarity_matrix(nii_files, rsn_files)
similarities_rsns_df

100%|██████████| 403/403 [01:23<00:00,  4.82it/s]


,sub,sbm,pi_0,pi_1,pi_2,pi_3,pi_4,pi_5,pi_6,pi_7,pi_8,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.38600489497184753, 0.0, 0.0019676736555993...","[[0.5441304445266724, 0.0028255197685211897, 0...","[[0.2663203775882721, 0.0, 0.3876191973686218,...","[[0.3542596697807312, 0.46956807374954224, 0.0...","[[0.3612755835056305, 0.45853516459465027, 0.0...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,0.286725,0
1,SLC01,sbm-nd-h,"[[0.38189244270324707, 0.0, 0.1421464681625366...","[[0.5570521354675293, 0.004292700439691544, 0....","[[0.27577051520347595, 0.003161746310070157, 0...","[[0.3567929267883301, 0.4217322766780853, 0.0,...","[[0.3623256981372833, 0.29305899143218994, 0.0...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,0.254538,1
2,SLC01,sbm-nd-h,"[[0.37662267684936523, 0.0, 0.1439680755138397...","[[0.525486946105957, 0.0, 0.024157829582691193...","[[0.3446800708770752, 0.0, 0.4229114055633545,...","[[0.36024191975593567, 0.47476011514663696], [...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.102439,2
3,SLC01,sbm-nd-h,"[[0.3807685971260071, 0.0, 0.13954979181289673...","[[0.5065487623214722, 0.0099861491471529, 0.01...","[[0.3234710693359375, 0.0, 0.4613460898399353,...","[[0.35823509097099304, 0.507617175579071], [0....","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.079128,3
4,SLC01,sbm-nd-h,"[[0.3760620653629303, 0.0, 0.14012204110622406...","[[0.534360945224762, 0.009457714855670929, 0.1...","[[0.3356129825115204, 0.0, 0.4081932604312897,...","[[0.3582858443260193, 0.46574243903160095], [0...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.078369,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,SLC10,sbm-nd-h,"[[0.3807685971260071, 0.0, 0.13954979181289673...","[[0.5065487623214722, 0.0099861491471529, 0.01...","[[0.3234710693359375, 0.0, 0.4613460898399353,...","[[0.35823509097099304, 0.507617175579071], [0....","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.080736,3
70,SLC10,sbm-nd-h,"[[0.3760620653629303, 0.0, 0.14012204110622406...","[[0.534360945224762, 0.009457714855670929, 0.1...","[[0.3356129825115204, 0.0, 0.4081932604312897,...","[[0.3582858443260193, 0.46574243903160095], [0...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.052651,4
71,SLC10,sbm-nd-h,"[[0.5236111879348755, 0.0, 0.00087327050277963...","[[0.5469571948051453, 0.007281100377440453, 0....","[[0.2873786389827728, 0.0, 0.4216686189174652,...","[[0.35469910502433777, 0.508620023727417], [0....","[[0.3576491177082062, 0.554546058177948], [0.5...","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,0.035607,5
72,SLC10,sbm-nd-h,"[[0.3829512894153595, 0.0, 0.00109464733395725...","[[0.5684900879859924, 0.02039618231356144, 0.0...","[[0.33550533652305603, 0.0, 0.5089374780654907...","[[0.3560188412666321, 0.4746300280094147], [0....","[[0.36278781294822693], [0.5547022223472595], ...",NaN,NaN,NaN,NaN,0.017924,6


In [22]:
for sub, group in similarities_rsns_df.groupby('sub'):
    folder = f'{ESTIM_path}/individual/sub-{sub}/comparions-with-known-systems/{SBM}'
    os.system(f'mkdir -p {folder}')

    with open(f'{folder}/sys-rsns_desc-similarities.pkl', 'wb') as f:
        pickle.dump(group, f)

In [23]:
soft_similarities_rsns_df = get_soft_similarity_matrix(similarities_rsns_df, len(rsn_files))
soft_similarities_rsns_df

,sub,sbm,pi_0,pi_1,pi_2,pi_3,pi_4,pi_5,pi_6,pi_7,pi_8
0,SLC01,sbm-nd-h,"[[0.39316165210460957, 0.0, 0.0820441092960810...","[[0.5417366450140829, 0.005762634678763889, 0....","[[0.29723652853030585, 0.0008047851663297329, ...","[[0.3568245605998686, 0.4528253610302858, 0.0,...","[[0.36184990632443464, 0.24779787629973837, 0....","[[0.22366354609551026], [0.34198135011484987],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
1,SLC02,sbm-nd-h,"[[0.3922573825156949, 6.325632107867368e-08, 0...","[[0.5375447237589419, 0.00568090326945587, 0.0...","[[0.3036641541411454, 0.0005008802415544665, 0...","[[0.35708139395150806, 0.46192911606038217, 0....","[[0.361916473026583, 0.21412152660854444, 4.63...","[[0.18458109578538567], [0.28222432061150143],...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
2,SLC03,sbm-nd-h,"[[0.38328338701096887, 0.0, 0.0636084632610275...","[[0.54283423523446, 0.002938253791338818, 0.10...","[[0.2835790475813668, 0.0007512547043611388, 0...","[[0.3559949119595629, 0.4603316885026652, 0.0,...","[[0.36182839154696367, 0.32725291724418687, 0....","[[0.29002704854296746], [0.4434510824942165], ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
3,SLC04,sbm-nd-h,"[[0.3958861419022084, 0.0, 0.07985027403821704...","[[0.5384336304450036, 0.0048190745932050055, 0...","[[0.2959755636906624, 0.0005748054791707547, 0...","[[0.3565401102781296, 0.4697314917874337, 0.0,...","[[0.3616748767781258, 0.26522757076025016, 0.0...","[[0.2260748535168171], [0.3456682368779183], [...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
4,SLC05,sbm-nd-h,"[[0.3825436447070437, 0.0, 0.0751049991728539,...","[[0.5397286028968769, 0.00387347510851799, 0.1...","[[0.2903809091583437, 0.0006519442511408156, 0...","[[0.35634150917365137, 0.46383392562607867, 0....","[[0.36197063448380484, 0.2793185345426363, 0.0...","[[0.2479896221576524], [0.37917589737100155], ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
5,SLC06,sbm-nd-h,"[[0.40120724894972304, 2.807894338823662e-05, ...","[[0.5393750660871118, 0.006618171824465756, 0....","[[0.3103968718405318, 0.00042196766305483357, ...","[[0.3576406507402402, 0.45119303680939954, 0.0...","[[0.362020451951251, 0.1845782958321343, 0.001...","[[0.1784115603612074, 0.0], [0.272782090266338...","[[0.00962305741086915], [0.014713645665782475]...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
6,SLC07,sbm-nd-h,"[[0.38551731970866343, 0.0, 0.0712384287634062...","[[0.5401663723312187, 0.0039473733115831825, 0...","[[0.2894593850769998, 0.0006221113467185814, 0...","[[0.3562301195265888, 0.46471268352617773, 0.0...","[[0.36185987783317064, 0.2910378714239695, 0.0...","[[0.254480821659106], [0.3891009352599385], [0...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
7,SLC08,sbm-nd-h,"[[0.3950352072095995, 0.0, 0.08242876288752529...","[[0.540294731740736, 0.006071282711062172, 0.0...","[[0.30093524523244763, 0.000647016831378884, 0...","[[0.3567216414519487, 0.46925754892995325, 0.0...","[[0.3618219010008976, 0.22861522329232617, 0.0...","[[0.20043645814356148], [0.306467154638168], [...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]"
8,SLC09,sbm-nd-h,"[[0.39473361767134346, 0.0, 0.0736893843027374...","[[0.5405193556908939, 0.0059730795179506755, 0...","[[0.299673452037947, 0.0004989769166672095, 0....","[[0.35653023649601395, 0.47050131846670623, 0....","[

In [24]:
folder = f'{ESTIM_path}/group/comparisons-with-known-systems/{SBM}'
os.system(f'mkdir -p {folder}')

for col in cols:

    ncols = 1
    nrows = len(soft_similarities_rsns_df)
    fig, axs = plt.subplots(nrows, ncols, figsize=(20*ncols, 6*nrows),)
    fig.tight_layout(h_pad=3, w_pad=3)
    fig.suptitle(f'{SBM} {col}', x=0.0, y=1.0)
    for idx, row in soft_similarities_rsns_df.iterrows():
        r = idx
        ax = axs[r]
        sub = row['sub']
        soft_X = row[col]
        sns.heatmap(soft_X, ax=ax, vmin=0.0, vmax=1.0, annot=True, fmt='.2f')
        ax.set(xlabel=f'comm', ylabel=f'ic', title=f'{sub}: similarity matrix')

    fig.savefig(f'{folder}/col-{col}_sys-rsn_desc-similarity-matrices.pdf', bbox_inches='tight')
    plt.close('all')

---

groups of comms that resemble groups of ics/rsns

In [25]:
def selected_comms_in_group(Xs, thresh=0.1):
    sel_comms = []
    for X in Xs:
        scs = np.sum(X, axis=0) > thresh
        scs = np.where(scs)[0]
        sel_comms += [scs]
    lens = list(map(len, sel_comms))
    loc = np.where(lens == np.min(lens))[0][0]
    sel_comms = sel_comms[loc]
    return sel_comms

In [26]:
def align_labels(labels_to_align, reference_labels, num_clusters):
    """
    Align labels_to_align with reference_labels using Hungarian algorithm.
    """
    cost_matrix = np.zeros((num_clusters, num_clusters))
    for i in range(num_clusters):
        for j in range(num_clusters):
            mask_i = labels_to_align == i
            mask_j = reference_labels == j
            cost_matrix[i,j] = -np.sum(mask_i & mask_j)

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    aligned_labels = np.zeros_like(labels_to_align)
    for old_label, new_label in zip(row_ind, col_ind):
        aligned_labels[labels_to_align == old_label] = new_label
    return aligned_labels

def compute_block_matrix(matrices, row_labels, col_labels, num_clusters):
    """
    Compute block matrix
    
    Args:
        matrices: List of matrices
        row_labels: Row cluster labels
        col_labels: Column cluster labels
        n_clusters: Number of clusters
    Returns:
        block_matrix: Average values within each block (n_clusters, n_clusters)
    """
    block_matrix = np.zeros((num_clusters, num_clusters))
    for i in range(num_clusters):
        for j in range(num_clusters):
            row_mask = (row_labels == i)
            col_mask = (col_labels == j)
            block_matrix[i,j] = np.mean([mat[row_mask][:, col_mask] for mat in matrices])
    return block_matrix

def consensus_coclustering(matrices, num_clusters, compute_block_pattern=False):
    """
    Get consensus labels, with optional block pattern computation.
    Only compute block pattern for reference labels, skip for bootstrap.
    """
    # Get individual coclusterings
    models = [SpectralCoclustering(n_clusters=num_clusters).fit(mat) for mat in matrices]

    # Build consensus matrices
    n_rows, n_cols = matrices[0].shape
    row_labels = [model.row_labels_ for model in models]
    col_labels = [model.column_labels_ for model in models]

    row_consensus = np.mean([np.eye(n_rows)[labels][:, labels] for labels in row_labels], axis=0)
    col_consensus = np.mean([np.eye(n_cols)[labels][:, labels] for labels in col_labels], axis=0)

    # Get consensus labels
    final_rows = SpectralClustering(n_clusters=num_clusters, affinity='precomputed').fit_predict(row_consensus)
    final_cols = SpectralClustering(n_clusters=num_clusters, affinity='precomputed').fit_predict(col_consensus)

    if compute_block_pattern:
        # Compute block pattern only for reference
        block_matrix = compute_block_matrix(matrices, final_rows, final_cols, num_clusters)
        return final_rows, final_cols, block_matrix
    
    return final_rows, final_cols

def align_row_col_labels(row_labels, col_labels, block_pattern, n_clusters):
    """
    Align column labels to row labels based on block pattern.
    """
    _, col_ind = linear_sum_assignment(-block_pattern)
    col_mapping = np.zeros_like(col_ind)
    col_mapping[col_ind] = np.arange(n_clusters)
    aligned_cols = col_mapping[col_labels]

    return row_labels, aligned_cols

def compute_element_stability(ref_labels, boot_labels_list):
    """
    Compute stability of cluster assignment for each element.

    Args:
        ref_labels: Reference labels
        boot_labels_list: List of labels from bootstrap samples
    Returns:
        element_stability: Array of stability scores for each element
    """
    boot_labels = np.array(boot_labels_list)
    matches = (boot_labels == ref_labels[np.newaxis, :])
    element_stability = np.mean(matches, axis=0)
    return element_stability

def bootstrap_consensus(matrices, num_clusters, n_bootstrap=100):
    """
    Bootstrap with stability analysis.

    Returns:
        ref_rows: Reference row labels
        ref_cols: Reference column labels
        mean_pattern: Mean block pattern
        ci_<lower, upper>: 95% CI around mean_pattern
        row_stability: Stability matrix for rows
        col_stability: Stability matrix for columns
        row_element_stability: Individual stability for each row
        col_element_stability: Individual stability for each column
    """
    ref_rows, ref_cols, ref_pattern = consensus_coclustering(matrices, num_clusters, compute_block_pattern=True)
    ref_rows, ref_cols = align_row_col_labels(ref_rows, ref_cols, ref_pattern, num_clusters)

    n_rows, n_cols = matrices[0].shape
    row_stability = np.zeros((n_rows, n_rows))
    col_stability = np.zeros((n_cols, n_cols))
    
    boot_row_labels = []
    boot_col_labels = []
    block_patterns = []

    for _ in tqdm(range(n_bootstrap)):
        indices = np.random.choice(len(matrices), size=len(matrices), replace=True)
        boot_matrices = [matrices[i] for i in indices]
        
        boot_rows, boot_cols = consensus_coclustering(boot_matrices, num_clusters, compute_block_pattern=False)
        aligned_rows = align_labels(boot_rows, ref_rows, num_clusters)
        aligned_cols = align_labels(boot_cols, ref_cols, num_clusters)
        
        # co-occurrence matrix
        row_stability += np.eye(n_rows)[aligned_rows][:, aligned_rows]
        col_stability += np.eye(n_cols)[aligned_cols][:, aligned_cols]
        
        boot_row_labels.append(aligned_rows)
        boot_col_labels.append(aligned_cols)
        
        block_matrix = compute_block_matrix(boot_matrices, aligned_rows, aligned_cols, num_clusters)
        block_patterns.append(block_matrix)
    
    row_stability /= n_bootstrap
    col_stability /= n_bootstrap

    row_element_stability = compute_element_stability(ref_rows, boot_row_labels)
    col_element_stability = compute_element_stability(ref_cols, boot_col_labels)

    mean_pattern = np.mean(block_patterns, axis=0)
    ci_lower = np.percentile(block_patterns, 2.5, axis=0)
    ci_upper = np.percentile(block_patterns, 97.5, axis=0)

    return (
        ref_rows, ref_cols, mean_pattern, ci_lower, ci_upper, 
        row_stability, col_stability, 
        row_element_stability, col_element_stability,
    )

In [27]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.inputs.outputtype = 'NIFTI'
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

In [28]:
# save ic/rsn groups
def group_icimg(ic_files, ics):
        icvols = []
        for ic in ics:
            icimg =ants.image_read(ic_files[ic]) 
            icvols += [icimg.numpy()]
        grp_icvol = np.sum(np.stack(icvols, axis=-1), axis=-1)
        grp_icvol = (grp_icvol > 0).astype(np.float32)
        return icimg.new_image_like(grp_icvol)

def save_sys_groups(ic_files, num_clusters, row_labels, sys_name='ic'):
    for i in range(num_clusters):
        ics = np.where(row_labels == i)[0]
        grp_icimg = group_icimg(ic_files, ics)
        grp_icimg.to_file(f'{folder}/group-{i:02d}.nii.gz')

    in_files = sorted(glob.glob(f'{folder}/group-*.nii.gz'))
    out_file = f'{folder}/sys-{sys_name}_desc-sys-groups.nii.gz'
    concatenate(in_files, out_file)
    return None

# save comm groups
def save_comm_groups(num_clusters, sel_comms, col_labels, sys_name='ic'):

    cmask_img = ants.image_read(
        f'{BASE_path}/voxel/common_brain_mask.nii.gz'
    )

    col = cols[0]
    level = col.split('_')[-1]
    if level == 'aligned':
        name = f'mode-00'
    else:
        name = f'mode-00_level-{level}'

    # these comm files are only for visualization purposes
    comms_file = sorted(glob.glob(f'{ESTIM_path}/group/membership-mats-group-aligned/{SBM}/marginal-visuals/nii/{name}.nii.gz'))[0]
    comms_vol = ants.image_read(comms_file).numpy()

    for i in range(num_clusters):
        comms = sel_comms[col_labels == i]
        grp_comms_vol = np.sum(comms_vol[..., comms], axis=-1)
        cmask_img.new_image_like(grp_comms_vol).to_file(
            f'{folder}/group-{i:02d}.nii.gz'
        )
    in_files = sorted(glob.glob(f'{folder}/group-*.nii.gz'))
    out_file = f'{folder}/sys-{sys_name}_desc-comm-groups.nii.gz'
    concatenate(in_files, out_file)
    return None

In [32]:
# resemblance to ics
num_clusters=5

folder = f'{ESTIM_path}/group/comparisons-with-known-systems/{SBM}/nii/num-clusters-{num_clusters}'
os.system(f'mkdir -p {folder}')

sys_name = 'ic'
sys_names = [ic_file.split('/')[-1].split('-')[2] for ic_file in ic_files]
for col in cols[:2]:
    Xs = soft_similarities_rsns_df[col].to_list()
    sel_comms = selected_comms_in_group(Xs)
    Xs = [X[:, sel_comms] for X in Xs]

    (
        row_labels, col_labels, mean_pat, ci_lower, ci_upper,
        row_stability, col_stability,
        row_element_stability, col_element_stability,
    ) = bootstrap_consensus(matrices=Xs, num_clusters=num_clusters, n_bootstrap=100)

    # create nii files for the groups and then open them side-by-side
    save_sys_groups(rsn_files, num_clusters, row_labels, sys_name)
    save_comm_groups(num_clusters, sel_comms, col_labels, sys_name)

100%|██████████| 100/100 [00:09<00:00, 10.23it/s]


241230-00:33:56,826 nipype.interface INFO:
	 stderr 2024-12-30T00:33:56.826683:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:33:56,829 nipype.interface INFO:
	 stderr 2024-12-30T00:33:56.829412:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:33:56,830 nipype.interface INFO:
	 stderr 2024-12-30T00:33:56.829412:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:33:56,862 nipype.interface INFO:
	 stderr 2024-12-30T00:33:56.862270:++ elapsed time = 0.0 s
241230-00:33:57,281 nipype.interface INFO:
	 stderr 2024-12-30T00:33:57.281184:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:33:57,285 nipype.interface INFO:
	 stderr 2024-12-30T00:33:57.285319:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:33:57,287 nipype.interface INFO:
	 stderr 2024-12-30T00:33:57.287893:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:33:57,346 nipype.interface INFO:
	 stderr 2024-12

100%|██████████| 100/100 [00:09<00:00, 10.78it/s]


241230-00:34:07,160 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.160718:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:07,163 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.163805:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:07,165 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.165021:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:07,200 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.200202:++ elapsed time = 0.0 s
241230-00:34:07,600 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.600231:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:07,603 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.603009:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:07,604 nipype.interface INFO:
	 stderr 2024-12-30T00:34:07.604312:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:07,636 nipype.interface INFO:
	 stderr 2024-12

In [33]:
# resemblance to rsns
num_clusters=5

folder = f'{ESTIM_path}/group/comparisons-with-known-systems/{SBM}/nii/num-clusters-{num_clusters}'
os.system(f'mkdir -p {folder}')

sys_name = 'rsn'
sys_names = [rsn_file.split('/')[-1].split('-')[2] for rsn_file in rsn_files]
for col in cols[:2]:
    Xs = soft_similarities_rsns_df[col].to_list()
    sel_comms = selected_comms_in_group(Xs)
    Xs = [X[:, sel_comms] for X in Xs]

    (
        row_labels, col_labels, mean_pat, ci_lower, ci_upper,
        row_stability, col_stability,
        row_element_stability, col_element_stability,
    ) = bootstrap_consensus(matrices=Xs, num_clusters=num_clusters, n_bootstrap=100)

    # create nii files for the groups and then open them side-by-side
    save_sys_groups(rsn_files, num_clusters, row_labels, sys_name)
    save_comm_groups(num_clusters, sel_comms, col_labels, sys_name)

100%|██████████| 100/100 [00:09<00:00, 10.16it/s]


241230-00:34:18,37 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.037520:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:18,42 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.042298:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:18,43 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.043719:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:18,83 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.083055:++ elapsed time = 0.0 s
241230-00:34:18,497 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.496896:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:18,499 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.499906:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:18,500 nipype.interface INFO:
	 stderr 2024-12-30T00:34:18.500829:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:18,533 nipype.interface INFO:
	 stderr 2024-12-30T

100%|██████████| 100/100 [00:09<00:00, 10.89it/s]


241230-00:34:28,261 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.261580:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:28,265 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.265770:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:28,268 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.268394:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:28,323 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.323129:++ elapsed time = 0.1 s
241230-00:34:28,737 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.737476:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241230-00:34:28,741 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.741647:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241230-00:34:28,744 nipype.interface INFO:
	 stderr 2024-12-30T00:34:28.744149:*+ WARNING: Set TR of output dataset to 1.0 s
241230-00:34:28,787 nipype.interface INFO:
	 stderr 2024-12